# Algerian Forest Fires: How different weather conditions induce forest fires

# Introduction

Forest fires - a particular subtype of wildfires- are uncontrolled fires caused by nature or humans which endanger wildlife and disrupt the ecosystem. Located on the Mediterranean coast, Algeria has a hot and dry climate which increases the rate of evaporation, leaving plants drier than usual. Consequently, the occurrence of forest fires increases. It is thus primordial to combat this environmental issue to prevent global warming and further decimation of Algerian forests. By observing the dataset of the Bejaia region in Algeria, can we use the climate measurements to predict whether a future forest fire will occur in Bejaia? To make this prediction, we will use the Algerian Forest Fires dataset that contains numerical data on relative humidity, wind speed, rain, and temperature and use these variables as predictors to foresee the possible occurrence of forest fires in the Bejaia region in order to take the necessary precautions to prevent the fire propagation and protect the ecosystem.


# Preliminary Exploratory Data Analysis

- Demonstrate that the dataset can be read from the web into R 
- Clean and wrangle your data into a tidy format
- **Using only training data**, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
- **Using only training data**, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.


# Methods

- Explain how you will conduct either your data analysis and which variables/columns you will use. 
    
Using K-nearest neighbours classification, we will use the columns: RH (relative humidity), Ws (wind speed), Rain, and Temperature as predictors to determine if a forest fire occurred in Bejaia and Sidi-Bel Abbes regions.  

- Describe at least one way that you will visualize the results
 
 We can visualize the results using a scatter plot. 


# Expected Outcomes and Significance

- What do you expect to find?
    - absences: inverse relationship
    - extra educational support: more support = better grades?
    - study time: limited rate of return
    - internet access at home: internet access = better grades
    
- What impact could such findings have?
    
- What future questions could this lead to?
